<a href="https://colab.research.google.com/github/duper203/RAG_Techniques_with_upstage/blob/main/upstage/03_Reliable_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reliable-RAG 🏷️




## Key Components
1. Document Loading and Chunking:

  Web-based documents are loaded and split into smaller, manageable chunks to facilitate efficient vector encoding and retrieval.

  
2. Vectorstore Creation:

  Utilizes Chroma and Upstage embeddings to encode document chunks into a vector store, enabling efficient similarity-based retrieval.

3. Document Relevancy Check:

  Implements a relevance-checking mechanism using a language model to filter out non-relevant documents before answer generation.

4. Answer Generation:

  Employs a language model to generate concise answers based on the relevant documents retrieved.

5. Hallucination Detection:

  A dedicated hallucination detection step ensures that the generated answers are grounded in the retrieved documents, preventing the inclusion of unsupported or erroneous information.

6. Document Snippet Highlighting:

  The system identifies and highlights the specific document segments that were directly used to generate the answer, providing transparency and traceability.


## Method Details
* Document Relevancy Filtering:

  By using a binary relevancy score generated by a language model, only the most relevant documents are passed on to the answer generation phase, reducing noise and improving the quality of the final answer.

* Hallucination Check:

  Before finalizing the answer, the system checks for hallucinations by verifying that the generated content is fully supported by the retrieved documents.

* Snippet Highlighting:

  This feature enhances transparency by showing the exact segments from the retrieved documents that contributed to the final answer.
---



### Import libraries and environment variables


In [ ]:
! pip3 install -qU langchain-upstage langchain-community langchain-Chroma

In [7]:
import os
from google.colab import userdata

os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")

# Create Vectorstore


In [8]:
### Build Index
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_upstage import UpstageEmbeddings

# Set embeddings
embedding_model = UpstageEmbeddings(model="solar-embedding-1-large")

# Docs to index
urls = [
    "https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-2-reflection/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-3-tool-use/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-4-planning/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io"
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag",
    embedding=embedding_model,
)

retriever = vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={'k': 4}, # number of documents to retrieve
            )

## Question

In [9]:
question = "what are the differnt kind of agentic design patterns?"

## Retrieve docs

In [10]:
docs = retriever.invoke(question)

## Check what our doc looklike

In [11]:
print(f"Title: {docs[0].metadata['title']}\n\nSource: {docs[0].metadata['source']}\n\nContent: {docs[0].page_content}\n")

Title: Agentic Design Patterns Part 5, Multi-Agent Collaboration

Source: https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io

Content: Agentic Design Patterns Part 5, Multi-Agent Collaboration✨ New course! Enroll in  Retrieval Optimization: From Tokenization to Vector QuantizationExplore CoursesAI NewsletterThe BatchAndrew's LetterData PointsML ResearchBlogCommunityForumEventsAmbassadorsAmbassador SpotlightResourcesCompanyAboutCareersContactStart LearningWeekly IssuesAndrew's LettersData PointsML ResearchBusinessScienceAI & SocietyCultureHardwareAI CareersAboutSubscribeThe BatchLettersArticleAgentic Design Patterns Part 5, Multi-Agent Collaboration Prompting an LLM to play different roles for different parts of a complex task summons a team of AI agents that can do the job more effectively.LettersTechnical InsightsPublishedApr 17, 2024Reading time3 min readShareDear friends,Multi-agent collaboration is the las

## Check document relevancy


In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_upstage import ChatUpstage

# LLM with function call
llm = ChatUpstage(model="solar-pro")

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question.
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals.
    Give a binary score 'yes' or 'no' to indicate whether the document is relevant to the question.
    Respond with only 'yes' or 'no'."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | llm

## Filter out the non-relevant docs

In [17]:
docs_to_use = []
for doc in docs:
    print(doc.page_content, '\n', '-'*50)
    res = retrieval_grader.invoke({"question": question, "document": doc.page_content})
    score = res.content.strip().lower()
    print(f"Relevance: {score}\n")
    if score == 'yes':
        docs_to_use.append(doc)

Agentic Design Patterns Part 5, Multi-Agent Collaboration✨ New course! Enroll in  Retrieval Optimization: From Tokenization to Vector QuantizationExplore CoursesAI NewsletterThe BatchAndrew's LetterData PointsML ResearchBlogCommunityForumEventsAmbassadorsAmbassador SpotlightResourcesCompanyAboutCareersContactStart LearningWeekly IssuesAndrew's LettersData PointsML ResearchBusinessScienceAI & SocietyCultureHardwareAI CareersAboutSubscribeThe BatchLettersArticleAgentic Design Patterns Part 5, Multi-Agent Collaboration Prompting an LLM to play different roles for different parts of a complex task summons a team of AI agents that can do the job more effectively.LettersTechnical InsightsPublishedApr 17, 2024Reading time3 min readShareDear friends,Multi-agent collaboration is the last of the four key AI agentic design patterns that I’ve described in recent letters. Given a complex task like writing software, a multi-agent approach would break down the task into subtasks to be executed by dif

## Generate Result

In [19]:
from langchain_core.output_parsers import StrOutputParser

# Prompt
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge.
Use three-to-five sentences maximum and keep the answer concise."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>"),
    ]
)

# LLM
llm = ChatUpstage(model="solar-pro")

# Post-processing
def format_docs(docs):
    return "\n".join(f"<doc{i+1}>:\nTitle:{doc.metadata['title']}\nSource:{doc.metadata['source']}\nContent:{doc.page_content}\n</doc{i+1}>\n" for i, doc in enumerate(docs))

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"documents":format_docs(docs_to_use), "question": question})
print(generation)

The four types of agentic design patterns are: reflection, tool use, planning, and multi-agent collaboration.


## Check for hallucinations

In [23]:
# GC
from langchain_upstage import UpstageGroundednessCheck

groundedness_check = UpstageGroundednessCheck()

gc_result = groundedness_check.invoke({"context": format_docs(docs_to_use), "answer": generation})

print(gc_result)
if gc_result.lower().startswith("grounded"):
    print("✅ Groundedness check passed")
else:
    print("❌ Groundedness check failed")

grounded
✅ Groundedness check passed


## Highlight used docs

In [27]:
from typing import List
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

# Data model
class HighlightDocuments(BaseModel):
    """Return the specific part of a document used for answering the question."""

    id: List[str] = Field(
        ...,
        description="List of id of docs used to answers the question"
    )

    title: List[str] = Field(
        ...,
        description="List of titles used to answers the question"
    )

    source: List[str] = Field(
        ...,
        description="List of sources used to answers the question"
    )

    segment: List[str] = Field(
        ...,
        description="List of direct segements from used documents that answers the question"
    )

# LLM
llm = ChatUpstage(model="solar-pro")

# parser
parser = PydanticOutputParser(pydantic_object=HighlightDocuments)

# Prompt
system = """You are an advanced assistant for document search and retrieval. You are provided with the following:
1. A question.
2. A generated answer based on the question.
3. A set of documents that were referenced in generating the answer.

Your task is to identify and extract the exact inline segments from the provided documents that directly correspond to the content used to
generate the given answer. The extracted segments must be verbatim snippets from the documents, ensuring a word-for-word match with the text
in the provided documents.

Ensure that:
- (Important) Each segment is an exact match to a part of the document and is fully contained within the document text.
- The relevance of each segment to the generated answer is clear and directly supports the answer provided.
- (Important) If you didn't used the specific document don't mention it.

Used documents: <docs>{documents}</docs> \n\n User question: <question>{question}</question> \n\n Generated answer: <answer>{generation}</answer>

<format_instruction>
{format_instructions}
</format_instruction>
"""


prompt = PromptTemplate(
    template= system,
    input_variables=["documents", "question", "generation"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Chain
doc_lookup = prompt | llm | parser

# Run
lookup_response = doc_lookup.invoke({"documents":format_docs(docs_to_use), "question": question, "generation": generation})

In [30]:
for id, title, source, segment in zip(lookup_response.id, lookup_response.title, lookup_response.source, lookup_response.segment):
    print(f"ID: {id}\nTitle: {title}\nSource: {source}\nText Segment: {segment}\n")

ID: doc4
Title: Four AI Agent Strategies That Improve GPT-4 and GPT-3.5 Performance
Source: https://www.deeplearning.ai/the-batch/how-agentic-can-improve-llm-performance/?ref=dl-staging-website.ghost.io
Text Segment: Four AI Agent Strigies That Improve GPT-4 and GPT-3.5 Performance

